# Prompting Capabilities 
Prompt the mistral models via API calls and perform various tasks like classification information extraction, personalization and summarization.

In [1]:
# !pip install mistralai

In [2]:
from helper import load_mistral_api_key
load_mistral_api_key()

In [3]:
from helper import mistral
mistral("hello, what can you do?")

'Hello! I can assist you with a variety of tasks. I can answer questions, provide information, set reminders, manage your schedule, send messages, and much more. I can also help you learn new things, like a new language or a new skill. How can I assist you today?'

## Classification

In [4]:
prompt = """
    You are a bank customer service bot. 
    Your task is to assess customer intent and categorize customer 
    inquiry after <<<>>> into one of the following predefined categories:
    
    card arrival
    change pin
    exchange rate
    country support 
    cancel transfer
    charge dispute
    
    If the text doesn't fit into any of the above categories, 
    classify it as:
    customer service
    
    You will only respond with the predefined category. 
    Do not provide explanations or notes. 
    
    ###
    Here are some examples:
    
    Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?
    Category: card arrival
    Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fees for foreign transactions.
    Category: exchange rate 
    Inquiry: What countries are getting support? I will be traveling and living abroad for an extended period of time, specifically in France and Germany, and would appreciate any information regarding compatibility and functionality in these regions.
    Category: country support
    Inquiry: Can I get help starting my computer? I am having difficulty starting my computer, and would appreciate your expertise in helping me troubleshoot the issue. 
    Category: customer service
    ###
    
    <<<
    Inquiry: {inquiry}
    >>>
    Category:
"""

In the above prompt our task is to assess customer intent and categorize customer inquiry. We have a list of predefined categories. If the text doesn't fit in any of the categories, classify it as customer service. In the above prompt we first assign a **role play** to the model as a `bank customer service bot` this adds personal context to the model. Next we used **few shot learning** where we give some exapmples in the prompts. This can improve model's performance, especially when the task is difficult or when we want the model to respond in a specific manner.
We use demminetors like '###' and '<<<' to specify the boundary between diferent sections of the text. In our example, we use the '###' to indicate examples and '<<<' to indicate customer inquiry. Finally in a case when the model is **verbose**, we can add: "do not provide explanations or notes", to make sure the output is concise.

#### If we want to make sure that our prompt doesn't have any grammetical mistake, ask Mistral to check the spelling and grammar of the prompt

In [5]:
response = mistral(f"Please correct the spelling and grammar of \
this prompt and return a text that is the same prompt,\
with the spelling and grammar fixed: {prompt}")

In [6]:
print(response)

You are a bank customer service bot.
Your task is to assess customer intent and categorize the customer inquiry following the "<<< >>>" into one of the following predefined categories:

card arrival
change pin
exchange rate
country support
cancel transfer
charge dispute

If the text doesn't fit into any of the above categories, classify it as:
customer service

You will only respond with the predefined category. Do not provide explanations or notes.

###
Here are some examples:

Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?
Category: card arrival

Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated 

Now using the corrected prompt let's try out the model by passing the inquiry which will enter in `{inquiry}`.

In [7]:
mistral(
    response.format(
        inquiry="I am inquiring about the availability of your cards in the EU"
    )
)

'country support'

## Information Extraction with JSON Mode

In [8]:
medical_notes = """
A 60-year-old male patient, Mr. Johnson, presented with symptoms
of increased thirst, frequent urination, fatigue, and unexplained
weight loss. Upon evaluation, he was diagnosed with diabetes,
confirmed by elevated blood sugar levels. Mr. Johnson's weight
is 210 lbs. He has been prescribed Metformin to be taken twice daily
with meals. It was noted during the consultation that the patient is
a current smoker. 
"""

The above text is of some medical information and we would like to extract some information from this text.

In [9]:
prompt = f"""
Extract information from the following medical notes:
{medical_notes}

Return json format with the following JSON schema: 

{{
        "age": {{
            "type": "integer"
        }},
        "gender": {{
            "type": "string",
            "enum": ["male", "female", "other"]
        }},
        "diagnosis": {{
            "type": "string",
            "enum": ["migraine", "diabetes", "arthritis", "acne"]
        }},
        "weight": {{
            "type": "integer"
        }},
        "smoking": {{
            "type": "string",
            "enum": ["yes", "no"]
        }}
}}
"""

In this prompt, we provide the medical notes and ask the model to return JSON format with the following JSON schema, where we define what we want to extract, the type of this variable and the list of output options. So for diagnosis, the model should output one of the four options provided in diagnosis section.
In this promt we explicitly ask to return JSON format. It is important to ask for the JSON format when we enable the JSON mode. Another strategy we use here, is that we define thte JSON schema. We use this JSON schema in the prompt to ensure the consistency and structure of the JSON output. Note that if we don't have the is_JSON=True the output may still be a JSON format, but it is recommended to enable the JSON mode to return a reliable JSON format.

In [10]:
response = mistral(prompt, is_json=True)
print(response)

{
        "age": 60,
        "gender": "male",
        "diagnosis": "diabetes",
        "weight": 210,
        "smoking": "yes"
}


## Personalization
LLMs are really good at personalization tasks, let's see how the model can create personalized email responses to address customer questions.

In [11]:
email = """
Dear mortgage lender, 

What's your 30-year fixed-rate APR, how is it compared to the 15-year 
fixed rate?

Regards,
Anna
"""

The above text is an email where the customer 'Anna' is asking the mortgage lender about the mortgage rate.

In [12]:
prompt = f"""

You are a mortgage lender customer service bot, and your task is to 
create personalized email responses to address customer questions.
Answer the customer's inquiry using the provided facts below. Ensure 
that your response is clear, concise, and directly addresses the 
customer's question. Address the customer in a friendly and 
professional manner. Sign the email with "Lender Customer Support."   
      
# Facts
30-year fixed-rate: interest rate 6.403%, APR 6.484%
20-year fixed-rate: interest rate 6.329%, APR 6.429%
15-year fixed-rate: interest rate 5.705%, APR 5.848%
10-year fixed-rate: interest rate 5.500%, APR 5.720%
7-year ARM: interest rate 7.011%, APR 7.660%
5-year ARM: interest rate 6.880%, APR 7.754%
3-year ARM: interest rate 6.125%, APR 7.204%
30-year fixed-rate FHA: interest rate 5.527%, APR 6.316%
30-year fixed-rate VA: interest rate 5.684%, APR 6.062%

# Email
{email}
"""

In the above prompt we have provided some numbers about the interest rates in the prompts. We use the string format to add the actual email content to this email variable at the end.

In [13]:
response = mistral(prompt)
print(response)

Subject: Re: Inquiry on 30-Year and 15-Year Fixed-Rate Loans

Dear Anna,

Thank you for reaching out to us with your inquiry. I'm delighted to help!

Our current 30-year fixed-rate mortgage carries an APR of 6.484%. When comparing this with our 15-year fixed-rate mortgage, you'll find that the APR for the 15-year term is lower at 5.848%. This difference is due to the shorter loan term of the 15-year fixed-rate mortgage, which translates to less interest paid over time.

Please let me know if you have any further questions or if you would like additional information on our loan options.

Best regards,
Lender Customer Support


We can see that we get a personalized email to Anna answering her questions based on the facts provided. With this kind of prompt, we can easily create our own customer service bot. Answer questions about our product. It is important to use clear and concise language when presenting these facts or your product information. This can help the model to provide accurate and quick responses to customer queries.

## Summarization

- We'll use this [article](https://www.deeplearning.ai/the-batch/mistral-enhances-ai-landscape-in-europe-with-microsoft-partnership-and-new-language-models) from The Batch. Summarization is a common task for LLMs, and Mistral model can do a really good job as summarization.

In [14]:
newsletter = """
European AI champion Mistral AI unveiled new large language models and formed an alliance with Microsoft. 

What’s new: Mistral AI introduced two closed models, Mistral Large and Mistral Small (joining Mistral Medium, which debuted quietly late last year). Microsoft invested $16.3 million in the French startup, and it agreed to distribute Mistral Large on its Azure platform and let Mistral AI use Azure computing infrastructure. Mistral AI makes the new models available to try for free here and to use on its La Plateforme and via custom deployments.

Model specs: The new models’ parameter counts, architectures, and training methods are undisclosed. Like the earlier, open source Mistral 7B and Mixtral 8x7B, they can process 32,000 tokens of input context. 

Mistral Large achieved 81.2 percent on the MMLU benchmark, outperforming Anthropic’s Claude 2, Google’s Gemini Pro, and Meta’s Llama 2 70B, though falling short of GPT-4. Mistral Small, which is optimized for latency and cost, achieved 72.2 percent on MMLU.
Both models are fluent in French, German, Spanish, and Italian. They’re trained for function calling and JSON-format output.
Microsoft’s investment in Mistral AI is significant but tiny compared to its $13 billion stake in OpenAI and Google and Amazon’s investments in Anthropic, which amount to $2 billion and $4 billion respectively.
Mistral AI and Microsoft will collaborate to train bespoke models for customers including European governments.
Behind the news: Mistral AI was founded in early 2023 by engineers from Google and Meta. The French government has touted the company as a home-grown competitor to U.S.-based leaders like OpenAI. France’s representatives in the European Commission argued on Mistral’s behalf to loosen the European Union’s AI Act oversight on powerful AI models. 

Yes, but: Mistral AI’s partnership with Microsoft has divided European lawmakers and regulators. The European Commission, which already was investigating Microsoft’s agreement with OpenAI for potential breaches of antitrust law, plans to investigate the new partnership as well. Members of President Emmanuel Macron’s Renaissance party criticized the deal’s potential to give a U.S. company access to European users’ data. However, other French lawmakers support the relationship.

Why it matters: The partnership between Mistral AI and Microsoft gives the startup crucial processing power for training large models and greater access to potential customers around the world. It gives the tech giant greater access to the European market. And it gives Azure customers access to a high-performance model that’s tailored to Europe’s unique regulatory environment.

We’re thinking: Mistral AI has made impressive progress in a short time, especially relative to the resources at its disposal as a startup. Its partnership with a leading hyperscaler is a sign of the tremendous processing and distribution power that remains concentrated in the large, U.S.-headquartered cloud companies.
"""

In [15]:
prompt = f"""
You are a commentator. Your task is to write a report on a newsletter. 
When presented with the newsletter, come up with interesting questions to ask,
and answer each question. 
Afterward, combine all the information and write a report in the markdown
format. 

# Newsletter: 
{newsletter}

# Instructions: 
## Summarize:
In clear and concise language, summarize the key points and themes 
presented in the newsletter.

## Interesting Questions: 
Generate three distinct and thought-provoking questions that can be 
asked about the content of the newsletter. For each question:
- After "Q: ", describe the problem 
- After "A: ", provide a detailed explanation of the problem addressed 
in the question.
- Enclose the ultimate answer in <>.

## Write a analysis report
Using the summary and the answers to the interesting questions, 
create a comprehensive report in Markdown format. 
"""

We have given the `{newsletter}` and provided some instructions followed by some questions to be answered and lastly to write an analysis report of the article. These instructions help the model to think in each step and generate a more comprehensive report.

In [16]:
response = mistral(prompt)
print(response)

# Summary
European AI startup Mistral AI has introduced two new large language models, Mistral Large and Mistral Small, and formed a strategic alliance with Microsoft. The partnership involves Microsoft investing in Mistral AI, distributing Mistral Large on its Azure platform, and providing Azure computing infrastructure. Mistral AI's new models outperform several competitors on the MMLU benchmark, and both models are fluent in multiple European languages. The partnership has received mixed reactions from European lawmakers and regulators, with some expressing concerns over data access and others supporting the relationship.

# Interesting Questions
## Q: What are the specific advantages of Mistral AI's new language models compared to their competitors?
A: Mistral AI's new language models, Mistral Large and Mistral Small, have demonstrated superior performance on the MMLU benchmark compared to Anthropic's Claude 2, Google’s Gemini Pro, and Meta’s Llama 2 70B. However, the exact advanta

## The Mistral Python client
- Below is the helper function that we imported from helper.py and used in this notebook.
- For more details, check out the [Mistral AI API documentation](https://docs.mistral.ai/api/)
- To get our own Mistral AI API key to use on our own, we can create an account and go to the [console](https://console.mistral.ai/) to subscribe and create an API key.

In [17]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage   

def mistral(user_message, 
            model="mistral-small-latest",
            is_json=False):
    client = MistralClient(api_key=os.getenv("MISTRAL_API_KEY"))
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, 
            messages=messages,
            response_format={"type": "json_object"})
    else:
        chat_response = client.chat(
            model=model, 
            messages=messages)
        
    return chat_response.choices[0].message.content